## submission 1 (3e+14e)

In [9]:
import re
def postprocess_text(row):
    generated_answer = row['generated_answer']
    if '</think>' in generated_answer.lower():
        generated_answer_process =  generated_answer.lower().split('</think>')[-1].strip()
    elif 'kết luận' in generated_answer.lower():
        generated_answer_process = generated_answer.lower().split('kết luận')[-1].strip()

    if row['question_type'] != 'Tự luận': 
        # Chỉ giữ các chữ cái tiếng việt và không phải khoảng trắng (\s)".
        generated_answer_process = re.sub(r"[^a-zA-ZÀ-Ỷà-ỹĂăÂâÊêÔôƠơƯưĐđ\s]", "", generated_answer_process).strip().capitalize()

    generated_answer_process = generated_answer_process
    return generated_answer_process

In [10]:
import pandas as pd
pd.options.mode.chained_assignment = None  # Tắt SettingWithCopyWarning
data_qwen_3e = pd.read_csv('submit_data/results_qwen_3e.csv')
data_qwen_3e['generated_answer_process'] = data_qwen_3e.apply(postprocess_text, axis=1)
data_qwen_3e = data_qwen_3e.rename(columns={'generated_answer':'qwen_3e_generated_answer', 'generated_answer_process':'qwen_3e_generated_answer_process'})

data_qwen_14e = pd.read_csv('submit_data/results_qwen_14e.csv')
data_qwen_14e['generated_answer_process'] = data_qwen_14e.apply(postprocess_text, axis=1)
data_qwen_14e = data_qwen_14e.rename(columns={'generated_answer':'qwen_14e_generated_answer', 'generated_answer_process':'qwen_14e_generated_answer_process'})

merged_df = data_qwen_3e.merge(
    data_qwen_14e[['question_id', 'qwen_14e_generated_answer','qwen_14e_generated_answer_process']],
    on='question_id',
    how='left'
)

# merged_df = merged_df.drop(columns=["question_type", "text", "relevant_articles"])
merged_df.head(1)

,question_id,question_type,text,relevant_articles,system_prompt,prompt,messages,qwen_3e_generated_answer,qwen_3e_generated_answer_process,qwen_14e_generated_answer,qwen_14e_generated_answer_process
0,private_test_alquac25_4,Trắc nghiệm,yếu tố nào không phải là tiêu chí để phân loại...,['Hệ thống thông tin phục vụ giao dịch điện tử...,"Bạn là một trợ lý pháp lý Tiếng Việt, có nhiệm...","Dựa vào bối cảnh bên dưới, hãy phân tích kỹ tr...",[{'content': 'Bạn là một trợ lý pháp lý Tiếng ...,<think>\n\n1. Phân tích câu hỏi: Câu hỏi yêu c...,B,<think>\n\n1. Phân tích câu hỏi: Câu hỏi yêu c...,B


In [11]:
merged_df['answer'] = merged_df['qwen_3e_generated_answer_process']

In [12]:
for i in range(merged_df.shape[0]):
    if merged_df['qwen_3e_generated_answer_process'][i] != merged_df['qwen_14e_generated_answer_process'][i]:
        # print(i, merged_df['prompt'][i], '\n\n\n')
        if merged_df['question_type'][i] == 'Trắc nghiệm':
            merged_df.loc[i]['answer']  = merged_df.loc[i]['qwen_14e_generated_answer_process']
        elif merged_df['question_type'][i] == 'Đúng/Sai':
            merged_df.loc[i]['answer'] = merged_df.loc[i]['qwen_14e_generated_answer_process']
        elif merged_df['question_type'][i] == 'Tự luận':
            merged_df.loc[i]['answer'] = merged_df.loc[i]['qwen_14e_generated_answer_process']
        else:
            print('error')

In [13]:
merged_df.keys()

Index(['question_id', 'question_type', 'text', 'relevant_articles',
       'system_prompt', 'prompt', 'messages', 'qwen_3e_generated_answer',
       'qwen_3e_generated_answer_process', 'qwen_14e_generated_answer',
       'qwen_14e_generated_answer_process', 'answer'],
      dtype='object')

In [14]:
import pandas as pd
import re

# Tạo cột tạm để trích số cuối
merged_df['question_id_num'] = merged_df['question_id'].apply(lambda x: int(re.search(r'_(\d+)$', x).group(1)))
records = (
    merged_df.sort_values('question_id_num')[['question_id', 'answer']]
    .to_dict(orient='records')
)

# Ghi file JSON
import json
with open('submission1.json', 'w', encoding='utf-8') as f:
    json.dump(records, f, ensure_ascii=False, indent=4)

## submission2 (grpo1 2 3)

In [164]:
import re
def postprocess_text(row):
    generated_answer = row['generated_answer']
    if '</think>' in generated_answer.lower():
        generated_answer_process =  generated_answer.lower().split('</think>')[-1].strip()
    elif 'kết luận' in generated_answer.lower():
        generated_answer_process = generated_answer.lower().split('kết luận')[-1].strip()
    else:
        generated_answer_process = 'rỗng'
        print('error so generated_answer_process empty')

    if row['question_type'] != 'Tự luận': 
        # Chỉ giữ các chữ cái tiếng việt và không phải khoảng trắng (\s)".
        generated_answer_process = re.sub(r"[^a-zA-ZÀ-Ỷà-ỹĂăÂâÊêÔôƠơƯưĐđ\s]", "", generated_answer_process).strip().capitalize()
    
    if row['question_type'] == 'Trắc nghiệm':
        generated_answer_process = generated_answer_process[:1]
        if generated_answer_process not in ['A', 'B', 'C', 'D']:
            generated_answer_process = 'rỗng'
    elif row['question_type'] == 'Đúng/Sai':
        generated_answer_process = generated_answer_process[:4]
        if generated_answer_process not in ['Đúng', 'Sai']:
            generated_answer_process = 'rỗng'

    if generated_answer_process == '':
        generated_answer_process = 'rỗng'
        # print(generated_answer_process)

    return generated_answer_process

In [169]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [170]:
import pandas as pd
pd.options.mode.chained_assignment = None  # Tắt SettingWithCopyWarning
data_qwen_grpo1 = pd.read_csv('submit_data/results_qwen_grpo1.csv')
data_qwen_grpo1['generated_answer_process'] = data_qwen_grpo1.apply(postprocess_text, axis=1)
data_qwen_grpo1 = data_qwen_grpo1.rename(columns={'generated_answer':'qwen_grpo1_generated_answer', 'generated_answer_process':'qwen_grpo1_generated_answer_process'})

data_qwen_grpo2 = pd.read_csv('submit_data/results_qwen_grpo2.csv')
data_qwen_grpo2['generated_answer_process'] = data_qwen_grpo2.apply(postprocess_text, axis=1)
data_qwen_grpo2 = data_qwen_grpo2.rename(columns={'generated_answer':'qwen_grpo2_generated_answer', 'generated_answer_process':'qwen_grpo2_generated_answer_process'})

data_qwen_grpo3 = pd.read_csv('submit_data/results_qwen_grpo3.csv')
data_qwen_grpo3['generated_answer_process'] = data_qwen_grpo3.apply(postprocess_text, axis=1)
data_qwen_grpo3 = data_qwen_grpo3.rename(columns={'generated_answer':'qwen_grpo3_generated_answer', 'generated_answer_process':'qwen_grpo3_generated_answer_process'})

merged_df = data_qwen_grpo1.merge(
    data_qwen_grpo2[['question_id', 'qwen_grpo2_generated_answer','qwen_grpo2_generated_answer_process']],
    on='question_id',
    how='left'
).merge(
    data_qwen_grpo3[['question_id', 'qwen_grpo3_generated_answer','qwen_grpo3_generated_answer_process']],
    on='question_id',
    how='left'
)
merged_df.head(1)

error so generated_answer_process empty
error so generated_answer_process empty


,question_id,question_type,text,relevant_articles,system_prompt,prompt,messages,qwen_grpo1_generated_answer,qwen_grpo1_generated_answer_process,qwen_grpo2_generated_answer,qwen_grpo2_generated_answer_process,qwen_grpo3_generated_answer,qwen_grpo3_generated_answer_process
0,private_test_alquac25_51,Trắc nghiệm,Thời hạn kháng cáo bản án sơ thẩm của tòa án t...,['Thời hạn kháng cáo\n\n1. Thời hạn kháng cáo ...,"Bạn là một trợ lý pháp lý Tiếng Việt, có nhiệm...","Dựa vào bối cảnh bên dưới, hãy phân tích kỹ tr...",[{'content': 'Bạn là một trợ lý pháp lý Tiếng ...,<think>\n\n1. Phân tích câu hỏi: \nCâu hỏi yê...,A,<think>\n\n1. Phân tích câu hỏi: \nCâu hỏi yê...,A,<think>\n\n1. Phân tích câu hỏi: \nCâu hỏi yê...,A


In [ ]:
merged_df['answer'] = ''
for i in range(merged_df.shape[0]):
    # if merged_df['question_id'][i] == 'private_test_alquac25_31':
    answer_dict = {}
    for j in range(1,4):
        answer_temp = merged_df[f'qwen_grpo{j}_generated_answer_process'][i]
        if answer_temp not in answer_dict:
            answer_dict.update({answer_temp : 1})
        else:
            count = answer_dict[answer_temp] 
            answer_dict.update({answer_temp : count+1})

    if 'rỗng' in answer_dict:
        answer_dict.pop('rỗng')
    
    # Với câu trắc nghiệm, đúng/sai
    if merged_df['question_type'][i] == 'Trắc nghiệm' or merged_df['question_type'][i] == 'Đúng/Sai':
        if len(answer_dict)==1:
            #print('len(answer_dict)==1')
            merged_df['answer'][i] = list(answer_dict.keys())[0]
        elif len(answer_dict)==0: # ko có câu trả lời
            #print('len(answer_dict)==0')
            if merged_df['question_type'][i] == 'Trắc nghiệm':
                merged_df['answer'][i] = 'C'
            elif merged_df['question_type'][i] == 'Đúng/Sai':
                merged_df['answer'][i] = 'Đúng'
            else:
                print('error')
        else: # có nhiều câu trả lời khác nhau
            #print('len(answer_dict)>1')
            # 👉 Lấy câu trả lời có tần suất cao nhất
            best_answer = max(answer_dict.items(), key=lambda x: x[1])[0]
            merged_df['answer'][i] = best_answer
    else: #tự luận
        merged_df['answer'][i] = merged_df['qwen_grpo3_generated_answer_process'][i]

In [179]:
merged_df.keys()

Index(['question_id', 'question_type', 'text', 'relevant_articles',
       'system_prompt', 'prompt', 'messages', 'qwen_grpo1_generated_answer',
       'qwen_grpo1_generated_answer_process', 'qwen_grpo2_generated_answer',
       'qwen_grpo2_generated_answer_process', 'qwen_grpo3_generated_answer',
       'qwen_grpo3_generated_answer_process', 'answer'],
      dtype='object')

In [180]:
import pandas as pd
import re

# Tạo cột tạm để trích số cuối
merged_df['question_id_num'] = merged_df['question_id'].apply(lambda x: int(re.search(r'_(\d+)$', x).group(1)))
records = (
    merged_df.sort_values('question_id_num')[['question_id', 'answer']]
    .to_dict(orient='records')
)

# Ghi file JSON
import json
with open('submission2.json', 'w', encoding='utf-8') as f:
    json.dump(records, f, ensure_ascii=False, indent=4)

## submission3 (grpo1 2 3 + 14e)

In [ ]:
import re
def postprocess_text(row):
    generated_answer = row['generated_answer']
    if '</think>' in generated_answer.lower():
        generated_answer_process =  generated_answer.lower().split('</think>')[-1].strip()
    elif 'kết luận' in generated_answer.lower():
        generated_answer_process = generated_answer.lower().split('kết luận')[-1].strip()
    else:
        generated_answer_process = 'rỗng'
        print('error so generated_answer_process empty')

    if row['question_type'] == 'Trắc nghiệm':
        generated_answer_process = re.sub(r"[^a-zA-ZÀ-Ỷà-ỹĂăÂâÊêÔôƠơƯưĐđ\s]", "", generated_answer_process).strip().capitalize()
        generated_answer_process = generated_answer_process[:1]
        if generated_answer_process not in ['A', 'B', 'C', 'D']:
            generated_answer_process = 'rỗng'
    elif row['question_type'] == 'Đúng/Sai':
        generated_answer_process = re.sub(r"[^a-zA-ZÀ-Ỷà-ỹĂăÂâÊêÔôƠơƯưĐđ\s]", "", generated_answer_process).strip().capitalize()
        generated_answer_process = generated_answer_process[:4]
        if generated_answer_process not in ['Đúng', 'Sai']:
            generated_answer_process = 'rỗng'
    else: 
        generated_answer_process = re.sub(r":", "", generated_answer_process)

    if generated_answer_process == '':
        generated_answer_process = 'rỗng'
        # print(generated_answer_process)

    return generated_answer_process

In [226]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [227]:
import pandas as pd
pd.options.mode.chained_assignment = None  # Tắt SettingWithCopyWarning
data_qwen_grpo1 = pd.read_csv('submit_data/results_qwen_grpo1.csv')
data_qwen_grpo1['generated_answer_process'] = data_qwen_grpo1.apply(postprocess_text, axis=1)
data_qwen_grpo1 = data_qwen_grpo1.rename(columns={'generated_answer':'qwen_grpo1_generated_answer', 'generated_answer_process':'qwen_grpo1_generated_answer_process'})

data_qwen_grpo2 = pd.read_csv('submit_data/results_qwen_grpo2.csv')
data_qwen_grpo2['generated_answer_process'] = data_qwen_grpo2.apply(postprocess_text, axis=1)
data_qwen_grpo2 = data_qwen_grpo2.rename(columns={'generated_answer':'qwen_grpo2_generated_answer', 'generated_answer_process':'qwen_grpo2_generated_answer_process'})

data_qwen_grpo3 = pd.read_csv('submit_data/results_qwen_grpo3.csv')
data_qwen_grpo3['generated_answer_process'] = data_qwen_grpo3.apply(postprocess_text, axis=1)
data_qwen_grpo3 = data_qwen_grpo3.rename(columns={'generated_answer':'qwen_grpo3_generated_answer', 'generated_answer_process':'qwen_grpo3_generated_answer_process'})

data_qwen_14e = pd.read_csv('submit_data/results_qwen_14e.csv')
data_qwen_14e['generated_answer_process'] = data_qwen_14e.apply(postprocess_text, axis=1)
data_qwen_14e = data_qwen_14e.rename(columns={'generated_answer':'qwen_14e_generated_answer', 'generated_answer_process':'qwen_14e_generated_answer_process'})

data_qwen_3e = pd.read_csv('submit_data/results_qwen_3e.csv')
data_qwen_3e['generated_answer_process'] = data_qwen_3e.apply(postprocess_text, axis=1)
data_qwen_3e = data_qwen_3e.rename(columns={'generated_answer':'qwen_3e_generated_answer', 'generated_answer_process':'qwen_3e_generated_answer_process'})


merged_df = data_qwen_grpo1.merge(
    data_qwen_grpo2[['question_id', 'qwen_grpo2_generated_answer','qwen_grpo2_generated_answer_process']],
    on='question_id',
    how='left'
).merge(
    data_qwen_grpo3[['question_id', 'qwen_grpo3_generated_answer','qwen_grpo3_generated_answer_process']],
    on='question_id',
    how='left'
).merge(
    data_qwen_14e[['question_id', 'qwen_14e_generated_answer','qwen_14e_generated_answer_process']],
    on='question_id',
    how='left'
).merge(
    data_qwen_3e[['question_id', 'qwen_3e_generated_answer','qwen_3e_generated_answer_process']],
    on='question_id',
    how='left'
)

merged_df.head(1)

error so generated_answer_process empty
error so generated_answer_process empty


,question_id,question_type,text,relevant_articles,system_prompt,prompt,messages,qwen_grpo1_generated_answer,qwen_grpo1_generated_answer_process,qwen_grpo2_generated_answer,qwen_grpo2_generated_answer_process,qwen_grpo3_generated_answer,qwen_grpo3_generated_answer_process,qwen_14e_generated_answer,qwen_14e_generated_answer_process,qwen_3e_generated_answer,qwen_3e_generated_answer_process
0,private_test_alquac25_51,Trắc nghiệm,Thời hạn kháng cáo bản án sơ thẩm của tòa án t...,['Thời hạn kháng cáo\n\n1. Thời hạn kháng cáo ...,"Bạn là một trợ lý pháp lý Tiếng Việt, có nhiệm...","Dựa vào bối cảnh bên dưới, hãy phân tích kỹ tr...",[{'content': 'Bạn là một trợ lý pháp lý Tiếng ...,<think>\n\n1. Phân tích câu hỏi: \nCâu hỏi yê...,A,<think>\n\n1. Phân tích câu hỏi: \nCâu hỏi yê...,A,<think>\n\n1. Phân tích câu hỏi: \nCâu hỏi yê...,A,<think>\n\n1. Phân tích câu hỏi: \nCâu hỏi yê...,A,<think>\n\n1. Phân tích câu hỏi: \nCâu hỏi yê...,A


In [ ]:
merged_df['answer'] = ''
for i in range(merged_df.shape[0]):
    # if merged_df['question_id'][i] == 'private_test_alquac25_31':
    answer_dict = {}
    for j in range(1,4):
        answer_temp = merged_df[f'qwen_grpo{j}_generated_answer_process'][i]
        if answer_temp not in answer_dict:
            answer_dict.update({answer_temp : 1})
        else:
            count = answer_dict[answer_temp] 
            answer_dict.update({answer_temp : count+1})

    # thêm qwen 14e
    answer_temp = merged_df[f'qwen_14e_generated_answer_process'][i]
    if answer_temp not in answer_dict:
        answer_dict.update({answer_temp : 1})
    else:
        count = answer_dict[answer_temp] 
        answer_dict.update({answer_temp : count+1})
    
    # sắp xếp dict để đưa câu trả lời có tần suất cao nahast lên trước
    answer_dict = dict(sorted(answer_dict.items(), key=lambda x: (x[1], x[0]), reverse=True))

    if 'rỗng' in answer_dict:
        answer_dict.pop('rỗng')
    
    # Với câu trắc nghiệm, đúng/sai
    if merged_df['question_type'][i] == 'Trắc nghiệm' or merged_df['question_type'][i] == 'Đúng/Sai':
        if len(answer_dict)==1:
            #print('len(answer_dict)==1')
            merged_df['answer'][i] = list(answer_dict.keys())[0]
        elif len(answer_dict)==0: # ko có câu trả lời
            #print('len(answer_dict)==0')
            if merged_df['question_type'][i] == 'Trắc nghiệm':
                merged_df['answer'][i] = 'C'
            elif merged_df['question_type'][i] == 'Đúng/Sai':
                merged_df['answer'][i] = 'Đúng'
            else:
                print('error')
        else: # có nhiều câu trả lời khác nhau
            #print('len(answer_dict)>1')
            # 👉 Lấy câu trả lời có tần suất cao nhất
            best_answer = max(answer_dict.items(), key=lambda x: x[1])[0]
            merged_df['answer'][i] = best_answer
    else: #tự luận
        merged_df['answer'][i] = merged_df['qwen_grpo3_generated_answer_process'][i]
        if merged_df['answer'][i] == '' or merged_df['answer'][i] == 'rỗng':
            merged_df['answer'][i] = merged_df['qwen_14e_generated_answer_process'][i]

In [ ]:
import pandas as pd
import re

# Tạo cột tạm để trích số cuối
merged_df['question_id_num'] = merged_df['question_id'].apply(lambda x: int(re.search(r'_(\d+)$', x).group(1)))
records = (
    merged_df.sort_values('question_id_num')[['question_id', 'answer']]
    .to_dict(orient='records')
)

# Ghi file JSON
import json
with open('submission3.json', 'w', encoding='utf-8') as f:
    json.dump(records, f, ensure_ascii=False, indent=4)